In [1]:
import os
core = os.cpu_count()
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Function that adds water to each glass equally for A and removes water from the cup with the most amount of water and the next cup clockwise.
def constant_div(n_1, steps_1): #n_1 = game size, steps_1 = number of steps for which the process is repeated. 
    tot = 0.5 #total amount of water available = 0.5 pints
    glasses = np.zeros(n_1) #Empty glassses 
    addition = tot / n_1 #Amount of water being added to each glass
    max_water = 0 #keeps track of the highest water level in each step
    overall_max = 0 #keeps track of the highest water level every achieved
    k = 0
    while k <= steps_1:
        if max_water <= 1:
            for i in range(n_1):
                glasses[i] += addition
            removal_index = glasses.argmax()
            if removal_index == n_1 - 1:
                glasses[removal_index] = 0
                glasses[0] = 0
            else:
                glasses[removal_index] = 0
                glasses[removal_index + 1] = 0
            max_water = glasses.max()
            if max_water >= overall_max:
                overall_max = max_water

            print("No glasses full. Max water: " + str(max_water))
            print(glasses)
        else:
            return overall_max
        k += 1
    print('Overall max water achieved = ', str(overall_max))
    return overall_max

In [3]:
constant_div(8,10)

No glasses full. Max water: 0.0625
[0.     0.     0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
No glasses full. Max water: 0.125
[0.0625 0.0625 0.     0.     0.125  0.125  0.125  0.125 ]
No glasses full. Max water: 0.1875
[0.125  0.125  0.0625 0.0625 0.     0.     0.1875 0.1875]
No glasses full. Max water: 0.1875
[0.1875 0.1875 0.125  0.125  0.0625 0.0625 0.     0.    ]
No glasses full. Max water: 0.1875
[0.     0.     0.1875 0.1875 0.125  0.125  0.0625 0.0625]
No glasses full. Max water: 0.1875
[0.0625 0.0625 0.     0.     0.1875 0.1875 0.125  0.125 ]
No glasses full. Max water: 0.1875
[0.125  0.125  0.0625 0.0625 0.     0.     0.1875 0.1875]
No glasses full. Max water: 0.1875
[0.1875 0.1875 0.125  0.125  0.0625 0.0625 0.     0.    ]
No glasses full. Max water: 0.1875
[0.     0.     0.1875 0.1875 0.125  0.125  0.0625 0.0625]
No glasses full. Max water: 0.1875
[0.0625 0.0625 0.     0.     0.1875 0.1875 0.125  0.125 ]
No glasses full. Max water: 0.1875
[0.125  0.125  0.0625 0.0625 0.     

0.1875

In [4]:
#Same function as above without the print statements
def constant_div(n_1, steps_1):
    tot = 0.5
    glasses = np.zeros(n_1)
    addition = tot / n_1
    max_water = 0
    overall_max = 0
    k = 0
    while k <= steps_1:
        if max_water <= 1:
            for i in range(n_1):
                glasses[i] += addition
            removal_index = glasses.argmax()
            if removal_index == n_1 - 1:
                glasses[removal_index] = 0
                glasses[0] = 0
            else:
                glasses[removal_index] = 0
                glasses[removal_index + 1] = 0
            max_water = glasses.max()
            if max_water >= overall_max:
                overall_max = max_water

            #print("No glasses full. Max water: " + str(max_water))
            #print(glasses)
        else:
            return overall_max
        k += 1
    #print('Overall max water achieved = ', str(overall_max))
    return overall_max

In [5]:
#Similar function but instead of adding water in each glasses, we add water in every other glass. Removal scheme remains the same.
def constant_div_2(n_2, steps_2):
    tot = 0.5
    glasses = np.zeros(n_2)
    addition = (tot * 2) / n_2
    max_water = 0
    overall_max = 0
    k = 0
    addition_index = np.arange(0, n_2, 2)
    while k <= steps_2:
        if max_water <= 1:
            for i in addition_index:
                glasses[i] += addition
            removal_index = glasses.argmax()
            if removal_index == n_2 - 1:
                glasses[removal_index] = 0
                glasses[0] = 0
            else:
                glasses[removal_index] = 0
                glasses[removal_index + 1] = 0
            max_water = glasses.max()
            if max_water >= overall_max:
                overall_max = max_water

            #print("No glasses full. Max water: " + str(max_water))
        else:
            return overall_max
        k += 1

        #print('Overall max water achieved = ', str(overall_max))
    return overall_max



In [6]:
n_s = [5, 10, 20, 40, 50, 80, 100, 150, 200, 500, 800, 1000]
steps = 200_000
waters_no_div = []
waters_div = []

In [7]:
import ipyparallel as ipp

In [8]:
c = ipp.Client()
dview = c[:]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/ipyparallel/util.py:212: RuntimeWarning: IPython could not determine IPs for Samarths-MBP-2: [Errno 8] nodename nor servname provided, or not known
  warnings.warn(


In [12]:
# Use `sync_imports` to import required modules on all engines
steps = 200_000
with dview.sync_imports():
    import numpy as np

dview.execute('import numpy as np')


importing numpy on engine(s)


<AsyncResult(execute): pending>

In [13]:
dview.push({"constant_div_2" : constant_div_2})
dview.push({'steps' : steps})

<AsyncResult(_push): pending>

In [14]:
# Scatter 'k' to all engines if you have different values for each or simply pass it as an argument for the same 'k' for all simulations
n_s =  [5, 10, 20, 50, 100, 150, 200, 500, 800, 1000]   # Example: same 'k' for all engines

results = dview.map_sync(lambda x: constant_div_2(x, steps), n_s)

In [15]:
dview.push({'constant_div' : constant_div})

<AsyncResult(_push): pending>

In [16]:
results_2 = dview.map_sync(lambda x: constant_div(x, steps), n_s)